In [1]:
import numpy as np
import pandas as pd
import csv
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import StackingClassifier
import category_encoders as ce

def scale(a):
    if(a<0):
        a=-a
    a/=np.power(10,np.floor(np.log10(a)))
    
    return a

In [2]:
train_df=pd.read_csv('train.csv')
test_df=pd.read_csv('test.csv')

temp_train_df=train_df.drop(columns=['Fault'])
temp_test_df=test_df.drop(columns=['Id'])

total_df=pd.concat([temp_train_df,temp_test_df],ignore_index=True)

# 'Report Number', 'Local Case Number', 'Person ID' are not useful
total_df.drop(columns=['Agency Name','Driverless Vehicle','Road Name','Cross-Street Name','Report Number', 'Local Case Number', 'Person ID','Vehicle Model','Vehicle Make','Vehicle ID'],inplace=True)

# dropping columns with to many NaN values
total_df.dropna(axis=1,thresh=35000,inplace=True)

# total_df['Agency Name'] = total_df['Agency Name'].str.replace("Montgomery County Police",'MONTGOMERY')
total_df['Driver Substance Abuse'] = total_df['Driver Substance Abuse'].str.replace("CONTRIBUTED",'PRESENT')
total_df['Driver Substance Abuse'] = total_df['Driver Substance Abuse'].str.replace("COMBINED SUBSTANCE",'COMBINATION')

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3427: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
# set(total_total_df[''])
# total_total_df[''].values_count(dropna=False)
# total_total_df['']=total_total_df[''].fillna(value)

In [4]:
# print(total_df['Equipment Problems'].value_counts(dropna=False))
total_df['Equipment Problems']=total_df['Equipment Problems'].fillna(value='NO MISUSE')
# total_df['Equipment Problems']=total_df['Equipment Problems'].astype('category').cat.codes

# print(total_df['Parked Vehicle'].value_counts(dropna=False))
# total_df['Parked Vehicle']=total_df['Parked Vehicle'].astype('category').cat.codes

# print(total_df['Vehicle Going Dir'].value_counts(dropna=False))
total_df['Vehicle Going Dir']=total_df['Vehicle Going Dir'].fillna('Unknown')
# total_df['Vehicle Going Dir']=total_df['Vehicle Going Dir'].astype('category').cat.codes

total_df['Vehicle Continuing Dir']=total_df['Vehicle Continuing Dir'].fillna('Unknown')
# total_df['Vehicle Continuing Dir']=total_df['Vehicle Continuing Dir'].astype('category').cat.codes

# print(total_df['Vehicle Movement'].value_counts(dropna=False))
total_df['Vehicle Movement']=total_df['Vehicle Movement'].fillna(value='UNKNOWN')
# total_df['Vehicle Movement']=total_df['Vehicle Movement'].astype('category').cat.codes

# print(total_df['Vehicle Body Type'].value_counts(dropna=False))
total_df['Vehicle Body Type']=total_df['Vehicle Body Type'].fillna(value='PASSENGER CAR')
# total_df['Vehicle Body Type']=total_df['Vehicle Body Type'].astype('category').cat.codes

# print(total_df['Vehicle Second Impact Location'].value_counts(dropna=False))
total_df['Vehicle Second Impact Location']=total_df['Vehicle Second Impact Location'].fillna(value='TWELVE OCLOCK')
# total_df['Vehicle Second Impact Location']=total_df['Vehicle Second Impact Location'].astype('category').cat.codes

total_df['Vehicle First Impact Location']=total_df['Vehicle First Impact Location'].fillna(value='TWELVE OCLOCK')
# total_df['Vehicle First Impact Location']=total_df['Vehicle First Impact Location'].astype('category').cat.codes

# print(total_df['Vehicle Damage Extent'].value_counts(dropna=False))
total_df['Vehicle Damage Extent']=total_df['Vehicle Damage Extent'].fillna(value='DISABLING')
# total_df['Vehicle Damage Extent']=total_df['Vehicle Damage Extent'].astype('category').cat.codes

# print(total_df['Drivers License State'].value_counts(dropna=False))
total_df['Drivers License State']=total_df['Drivers License State'].fillna(value='MD')
# total_df['Drivers License State']=total_df['Drivers License State'].astype('category').cat.codes

# total_df['Injury Severity']=total_df['Injury Severity'].astype('category').cat.codes

# print(total_df['Driver Substance Abuse'].value_counts(dropna=False))
total_df['Driver Substance Abuse']=total_df['Driver Substance Abuse'].fillna(value='UNKNOWN')
# total_df['Driver Substance Abuse']=total_df['Driver Substance Abuse'].astype('category').cat.codes

# print(total_df['Traffic Control'].value_counts(dropna=False))
total_df['Traffic Control']=total_df['Traffic Control'].fillna(value='UNKNOWN')
# total_df['Traffic Control']=total_df['Traffic Control'].astype('category').cat.codes

# print(total_df['Light'].value_counts(dropna=False))
total_df['Light']=total_df['Light'].fillna(value='UNKNOWN')
# total_df['Light']=total_df['Light'].astype('category').cat.codes

# print(total_df['Surface Condition'].value_counts(dropna=False))
total_df['Surface Condition']=total_df['Surface Condition'].fillna(value='UNKNOWN')
# total_df['Surface Condition']=total_df['Surface Condition'].astype('category').cat.codes

# print(total_df['Weather'].value_counts(dropna=False))
total_df['Weather']=total_df['Weather'].fillna(value='UNKNOWN')
# total_df['Weather']=total_df['Weather'].astype('category').cat.codes

# print(total_df['Collision Type'].value_counts(dropna=False))
total_df['Collision Type']=total_df['Collision Type'].fillna(value='SAME DIR REAR END')
# total_df['Collision Type']=total_df['Collision Type'].astype('category').cat.codes

# print(total_df['Cross-Street Type'].value_counts(dropna=False))
total_df['Cross-Street Type']=total_df['Cross-Street Type'].fillna(value='Unknown')
# total_df['Cross-Street Type']=total_df['Cross-Street Type'].astype('category').cat.codes

# print(total_df['Route Type'].value_counts(dropna=False))
total_df['Route Type']=total_df['Route Type'].fillna(value='Unknown')
# total_df['Route Type']=total_df['Route Type'].astype('category').cat.codes

# print(total_df['ACRS Report Type'].value_counts(dropna=False))
# total_df['ACRS Report Type']=total_df['ACRS Report Type'].astype('category').cat.codes

# print(total_df['Agency Name'].value_counts(dropna=False))
# total_df['Agency Name']=total_df['Agency Name'].astype('category').cat.codes

In [5]:
# total_df['Location'].value_counts(dropna=False)
total_df['Location']=total_df['Location'].str.replace(',',' ')

split_data = total_df["Location"].str.split(" ")
data = split_data.to_list()
for i in range(len(total_df.index)):
    data[i][1]=float(data[i][1])
    data[i][0]=float(data[i][0])
    data[i][1]=scale(data[i][1])

names = ["Location_1","Location_2"]

temp_df = pd.DataFrame(data, columns=names)

total_df=total_df.drop(columns=['Location'])
total_df=pd.concat([total_df,temp_df],axis=1)

total_df['Crash Date/Time'] = pd.to_datetime(total_df['Crash Date/Time'])
total_df['Hour'] = total_df['Crash Date/Time'].dt.hour
total_df['day'] = total_df['Crash Date/Time'].dt.dayofweek
total_df['year'] = total_df['Crash Date/Time'].dt.year
total_df['month']=total_df['Crash Date/Time'].dt.month
total_df.drop(columns=['Crash Date/Time'],inplace=True)

In [6]:
# one_hot = np.array(pd.get_dummies(total_df))
base_learners = [
                 ('rf_1', RandomForestClassifier(n_estimators=100, max_features='sqrt')),
                 ('rf_2', LGBMClassifier(boosting_type='dart', drop_rate=0.5, max_drop=50, n_estimators=1000)),
                 ('rf_3', XGBClassifier(use_label_encoder=False, learning_rate = 0.14, n_estimators=125, max_depth=8))
                ]

cl=[]
for column in total_df.columns:
    if total_df[column].dtype == object:
        cl.append(column)
encoder = ce.polynomial.PolynomialEncoder(verbose=False, cols=cl, drop_invariant=True)
one_hot= encoder.fit_transform(total_df)

clf = StackingClassifier(estimators=base_learners, n_jobs=12, final_estimator=LGBMClassifier(boosting_type='goss'))

train_labels=np.array(train_df['Fault'])
train_feas=one_hot[:51490]
test_feas=one_hot[51490:]

/home/cmaspi/.local/lib/python3.9/site-packages/category_encoders/polynomial.py:216: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X.drop(col, 1, inplace=True)


In [7]:
clf.fit(train_feas,train_labels)

StackingClassifier(estimators=[('rf_1',
                                RandomForestClassifier(max_features='sqrt')),
                               ('rf_2',
                                LGBMClassifier(boosting_type='dart',
                                               drop_rate=0.5, max_drop=50,
                                               n_estimators=1000)),
                               ('rf_3',
                                XGBClassifier(base_score=None, booster=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              enable_categorical=False,
                                              gamma=None, gpu_id=None,
                                              importance...
                                              min_child_weight=None,
                                 

In [8]:
predicn=clf.predict(test_feas)
predicn=np.array(predicn, dtype=int)
submsn = pd.DataFrame(predicn,columns=['Fault'])
submsn=pd.concat([test_df['Id'],submsn],axis=1)

submsn.to_csv('submsn.csv',index=False)
submsn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77235 entries, 0 to 77234
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Id      77235 non-null  int64
 1   Fault   77235 non-null  int64
dtypes: int64(2)
memory usage: 1.2 MB
